# Config

In [1]:
db_name = "energy"
db_user = "dawki"
db_pword_file = "C:\\Users\\dawki\\OneDrive\\Archive\\2019\\Documents\\private.txt"
with open(db_pword_file) as f:
    db_pword = f.read()

In [145]:
logger = logging.getLogger(__name__)

# Imports

In [151]:
import psycopg2
from psycopg2 import DatabaseError
import logging
import requests
from parser_opennem import parse_request_json
from db_populate import insert_batch

In [142]:
from data import ZONE_KEY_TO_REGION, REGION_LONG_NAMES, NETWORK_LONG_NAMES

# Main

In [150]:
## Params
region = ZONE_KEY_TO_REGION['AUS-NSW']
month_start = dt.datetime.today().date().replace(day=1)
url = f'http://api.opennem.org.au/stats/power/network/fueltech/NEM/{region}?month={month_start}'

## Request data
logger.info(f'Requesting {url}..')
r = requests.get(url)
r.raise_for_status()

## Parse data 
logger.debug('Parsing JSON..')
meta, price, power = parse_request_json(request_json)

## Connect to DB
logger.debug('Connect to DB..')
dbc = psycopg2.connect(user=db_user,
                        password=db_pword,
                        host="127.0.0.1",
                        port="5432",
                        database=db_name)
cur = dbc.cursor()

## Insert batch
logger.debug('Insert batch..')
insert_batch(cursor,logger,meta,price,power)

## Commit transaction
logger.debug('Commiting to DB..')
dbc.commit()

## Close connection
logger.debug('Close connection to DB..')
dbc.close()